In [1]:
#IST664 final project

import nltk
import random
import os
from nltk.corpus import stopwords
import sys
from nltk import FreqDist
'''
  This program shell reads email data for the spam classification problem.
  The input to the program is the path to the Email directory "corpus" and a limit number.
  The program reads the first limit number of ham emails and the first limit number of spam.
  It creates an "emaildocs" variable with a list of emails consisting of a pair
    with the list of tokenized words from the email and the label either spam or ham.
  It prints a one example emails.
 
'''

# define a feature definition function here


# function to read spam and ham files, train and test a classifier 
def processspamham(file_path):
  # convert the limit argument from a string to an int

  # start lists for spam and ham email texts
    hamtexts = []
    spamtexts = []
    os.chdir(file_path)
    print(os.getcwd())
  # process all files in directory that end in .txt up to the limit
  #    assuming that the emails are sufficiently randomized
    for file in os.listdir("./spam"):
        if (file.endswith(".txt")):
      # open file for reading and read entire file into a string
            f = open("./spam/"+file, 'r', encoding="latin-1")
            spamtexts.append (f.read())
            f.close()
    for file in os.listdir("./ham"):
        if (file.endswith(".txt")):
      # open file for reading and read entire file into a string
            f = open("./ham/"+file, 'r', encoding="latin-1")
            hamtexts.append (f.read())
            f.close()
  
  # print number emails read
    print ("Number of spam files:",len(spamtexts))
    print ("Number of ham files:",len(hamtexts))
#only take the equal value 
    if (len(spamtexts)>len(hamtexts)):
        length_mail=len(hamtexts)
    else:
        length_mail=len(spamtexts)
  # create list of mixed spam and ham email documents as (list of words, label)
    emaildocs = []
  # add all the spam
    for spam in spamtexts[:length_mail]:
        tokens = nltk.word_tokenize(spam)
        emaildocs.append((tokens, 'spam'))
  # add all the regular emails
    for ham in hamtexts[:length_mail]:
        tokens = nltk.word_tokenize(ham)
        emaildocs.append((tokens, 'ham'))
#check length of our data
    print(f'Total email for experiment :{len(emaildocs)}')
    # Set the seed value
    
    random.seed(123)
    # randomize the list
    random.shuffle(emaildocs)
  
  # print a few token lists
    for email in emaildocs[:1]:
        print (f'Example email {email}')
    return emaildocs

file_path = '/Users/ryanrichardson/Syr/NLP/FinalProject/corpus'
emaildocs=processspamham(file_path)

/Users/ryanrichardson/Syr/NLP/FinalProject/corpus
Number of spam files: 1500
Number of ham files: 3671
Total email for experiment :3000
Example email (['Subject', ':', 'accrued', 'thru', 'nov', '.', '19', 'daren', ',', 'please', 'take', 'a', 'look', 'at', 'the', 'attached', 'spreadsheet', ',', 'we', 'have', 'over', '500', ',', '000', 'mmbtu', 'as', 'unaccounted', 'for', '.', '.', '.', 'o', "'", 'neal', '3', '-', '9686'], 'ham')


In [2]:
#total words
all_words = [word.lower() for email in emaildocs for word in email[0]]
print(f'All words length {len(all_words)}')

All words length 682966


In [3]:
#freq distribution
Freq_Dist_words=nltk.FreqDist(all_words)

# get the 2000 most frequently appearing keywords in the corpus
word_items = Freq_Dist_words.most_common(2000)
word_features = [word for (word, freq) in word_items]   # just the words
print(word_features[:50])

['-', '.', ',', '/', ':', 'the', 'to', 'and', 'of', 'a', 'for', 'ect', '?', 'in', 'you', '@', 'this', 'subject', 'is', 'on', '=', 'i', "'", ')', '(', 'be', '!', 'your', 'hou', 'that', 'enron', 'with', 'we', 'from', 'have', '$', 'will', 's', 'are', 'or', 'as', 'at', ';', 'it', '3', 'not', '2000', 'com', '_', 'if']


In [4]:
def document_features(document, word_features):
	document_words = set(document)
	features = {}
	for word in word_features:
		features['V_%s' % word] = (word in document_words)
	return features

In [5]:
# get features sets for a document, including keyword features and category feature
featuresets = [(document_features(d, word_features), c) for (d,c) in emaildocs]
# the feature sets are 3000 words long
print(len(featuresets))
#look feturesets
print(featuresets[0])


3000
({'V_-': True, 'V_.': True, 'V_,': True, 'V_/': False, 'V_:': True, 'V_the': True, 'V_to': False, 'V_and': False, 'V_of': False, 'V_a': True, 'V_for': True, 'V_ect': False, 'V_?': False, 'V_in': False, 'V_you': False, 'V_@': False, 'V_this': False, 'V_subject': False, 'V_is': False, 'V_on': False, 'V_=': False, 'V_i': False, "V_'": True, 'V_)': False, 'V_(': False, 'V_be': False, 'V_!': False, 'V_your': False, 'V_hou': False, 'V_that': False, 'V_enron': False, 'V_with': False, 'V_we': True, 'V_from': False, 'V_have': True, 'V_$': False, 'V_will': False, 'V_s': False, 'V_are': False, 'V_or': False, 'V_as': True, 'V_at': True, 'V_;': False, 'V_it': False, 'V_3': True, 'V_not': False, 'V_2000': False, 'V_com': False, 'V__': False, 'V_if': False, 'V_by': False, 'V_please': True, 'V_``': False, 'V_1': False, 'V_2': False, 'V_00': False, 'V_our': False, 'V_me': False, 'V_%': False, 'V_|': False, 'V_all': False, 'V_gas': False, 'V_>': False, 'V_#': False, 'V_can': False, 'V_*': False, 'V

In [6]:
# training using naive Baysian classifier with a 90/10 split
train_set, test_set = featuresets[300:], featuresets[:300]
classifier = nltk.NaiveBayesClassifier.train(train_set)

In [187]:
# evaluate the accuracy of the classifier
print (f'Accuracy:{nltk.classify.accuracy(classifier, test_set)}')
# the accuracy result may vary since we randomized the documents

Accuracy:0.968


In [7]:
# show which features of classifier are most informative
print(classifier.show_most_informative_features(30))

Most Informative Features
             V_forwarded = True              ham : spam   =    223.6 : 1.0
                   V_hou = True              ham : spam   =    206.7 : 1.0
                   V_nom = True              ham : spam   =    130.9 : 1.0
                   V_ect = True              ham : spam   =    129.1 : 1.0
                  V_2001 = True              ham : spam   =     82.1 : 1.0
            V_nomination = True              ham : spam   =     78.0 : 1.0
                   V_bob = True              ham : spam   =     57.4 : 1.0
          V_prescription = True             spam : ham    =     56.8 : 1.0
                V_farmer = True              ham : spam   =     50.6 : 1.0
                  V_lisa = True              ham : spam   =     47.3 : 1.0
                 V_susan = True              ham : spam   =     47.3 : 1.0
                  V_2005 = True             spam : ham    =     45.3 : 1.0
                    V_cc = True              ham : spam   =     43.8 : 1.0

In [8]:
#method of cross validation
## cross-validation ##
# this function takes the number of folds, the feature sets
# it iterates over the folds, using different sections for training and testing in turn
#   it prints the accuracy for each fold and the average accuracy at the end
def cross_validation_accuracy(num_folds, featuresets):
    subset_size = int(len(featuresets)/num_folds)
    print('Each fold size:', subset_size)
    accuracy_list = []
    goldlist = []
    predictedlist = []
    # iterate over the folds
    for i in range(num_folds):
        test_this_round = featuresets[(i*subset_size):][:subset_size]
        train_this_round = featuresets[:(i*subset_size)] + featuresets[((i+1)*subset_size):]
        # train using train_this_round
        classifier = nltk.NaiveBayesClassifier.train(train_this_round)
        # evaluate against test_this_round and save accuracy
        accuracy_this_round = nltk.classify.accuracy(classifier, test_this_round)
#for Evaluation measures: precision,recall,F1 and confusion matrix:recall precision 
#we save our gold list and predicted list for every itiration so we can create our confusion matirx
        for (features, label) in test_this_round:
            goldlist.append(label)
            predictedlist.append(classifier.classify(features))
        print (i, accuracy_this_round)
        accuracy_list.append(accuracy_this_round)
    # find mean accuracy over all rounds
    print ('mean accuracy', sum(accuracy_list) / num_folds)
    return goldlist,predictedlist

In [9]:
# perform the cross-validation on the featuresets with word features and generate accuracy
num_folds = 5
goldlist,predictedlist=cross_validation_accuracy(num_folds, featuresets)

Each fold size: 600
0 0.97
1 0.96
2 0.9633333333333334
3 0.9533333333333334
4 0.9683333333333334
mean accuracy 0.9630000000000001


In [10]:
# look at the first 30 examples
print(goldlist[:30])
print(predictedlist[:30])

['ham', 'ham', 'spam', 'spam', 'ham', 'spam', 'ham', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'ham', 'ham', 'spam', 'ham', 'spam', 'spam', 'ham', 'spam', 'spam', 'ham', 'spam']
['ham', 'ham', 'spam', 'spam', 'ham', 'spam', 'ham', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'ham', 'ham', 'spam', 'ham', 'spam', 'spam', 'ham', 'spam', 'spam', 'ham', 'spam']


In [11]:
cm = nltk.ConfusionMatrix(goldlist, predictedlist)
print(cm.pretty_format(sort_by_count=True, truncate=9))

     |         s |
     |    h    p |
     |    a    a |
     |    m    m |
-----+-----------+
 ham |<1396> 104 |
spam |    7<1493>|
-----+-----------+
(row = reference; col = test)



In [12]:
# or show the results as percentages
print(cm.pretty_format(sort_by_count=True, show_percents=True, truncate=9))

     |             s |
     |      h      p |
     |      a      a |
     |      m      m |
-----+---------------+
 ham | <46.5%>  3.5% |
spam |   0.2% <49.8%>|
-----+---------------+
(row = reference; col = test)



In [14]:
# Function to compute precision, recall and F1 for each label
#  and for any number of labels
# Input: list of gold labels, list of predicted labels (in same order)
# Output:  prints precision, recall and F1 for each label
def eval_measures(gold, predicted):
    # get a list of labels
    labels = list(set(gold))
    # these lists have values for each label 
    recall_list = []
    precision_list = []
    F1_list = []
    for lab in labels:
        # for each label, compare gold and predicted lists and compute values
        TP = FP = FN = TN = 0
        for i, val in enumerate(gold):
            if val == lab and predicted[i] == lab:  TP += 1
            if val == lab and predicted[i] != lab:  FN += 1
            if val != lab and predicted[i] == lab:  FP += 1
            if val != lab and predicted[i] != lab:  TN += 1
        # use these to compute recall, precision, F1
        recall = TP / (TP + FP)
        precision = TP / (TP + FN)
        recall_list.append(recall)
        precision_list.append(precision)
        F1_list.append( 2 * (recall * precision) / (recall + precision))

    # the evaluation measures in a table with one row per label
    print('\tPrecision\tRecall\t\tF1')
    # print measures for each label
    for i, lab in enumerate(labels):
        print(lab, '\t', "{:10.3f}".format(precision_list[i]), \
          "{:10.3f}".format(recall_list[i]), "{:10.3f}".format(F1_list[i]))

# call the function with our data
eval_measures(goldlist, predictedlist)

	Precision	Recall		F1
spam 	      0.995      0.935      0.964
ham 	      0.931      0.995      0.962


In [15]:
#lets remove stopwords
stopwords = nltk.corpus.stopwords.words('english')
new_all_words_list=[word for word in all_words if word not in stopwords]
new_all_words=nltk.FreqDist(new_all_words_list)
new_words_items=new_all_words.most_common(2000)
word_features_withoutstopwords = [word for (word, freq) in new_words_items]
# get features sets for a document, including keyword features and category feature
featuresets_withoutStopwords = [(document_features(d, word_features_withoutstopwords), c) for (d,c) in emaildocs]
# the feature sets are 2000 words long
print(featuresets_withoutStopwords[1])

({'V_-': False, 'V_.': True, 'V_,': True, 'V_/': False, 'V_:': True, 'V_ect': False, 'V_?': True, 'V_@': False, 'V_subject': False, 'V_=': False, "V_'": False, 'V_)': False, 'V_(': False, 'V_!': False, 'V_hou': False, 'V_enron': False, 'V_$': False, 'V_;': False, 'V_3': False, 'V_2000': False, 'V_com': False, 'V__': False, 'V_please': False, 'V_``': False, 'V_1': False, 'V_2': False, 'V_00': False, 'V_%': False, 'V_|': False, 'V_gas': False, 'V_>': False, 'V_#': True, 'V_*': False, 'V_deal': True, 'V_e': False, 'V_10': False, 'V_http': False, 'V_0': False, 'V_000': False, 'V_meter': True, 'V_pm': False, 'V_4': False, 'V_hpl': False, 'V_cc': False, 'V_5': False, 'V_company': False, 'V_2001': False, 'V_new': False, 'V_may': False, 'V_thanks': False, 'V_information': False, 'V_get': False, 'V_price': False, 'V_11': False, 'V_01': False, 'V_daren': True, 'V_corp': False, 'V_need': False, 'V_7': False, 'V_know': False, 'V_us': False, 'V_www': False, 'V_&': False, 'V_time': False, 'V_email':

In [16]:
# training using naive Baysian classifier with a 90/10 split
train_set, test_set = featuresets_withoutStopwords[300:], featuresets_withoutStopwords[:300]
classifier = nltk.NaiveBayesClassifier.train(train_set)
# evaluate the accuracy of the classifier
print (f'Accuracy after removing Stop Words :{nltk.classify.accuracy(classifier, test_set)}')
# the accuracy result may vary since we randomized the documents
# show which features of classifier are most informative
print(classifier.show_most_informative_features(30))

Accuracy after removing Stop Words :0.9666666666666667
Most Informative Features
             V_forwarded = True              ham : spam   =    223.6 : 1.0
                   V_hou = True              ham : spam   =    206.7 : 1.0
                   V_nom = True              ham : spam   =    130.9 : 1.0
                   V_ect = True              ham : spam   =    129.1 : 1.0
                  V_2001 = True              ham : spam   =     82.1 : 1.0
            V_nomination = True              ham : spam   =     78.0 : 1.0
                   V_bob = True              ham : spam   =     57.4 : 1.0
          V_prescription = True             spam : ham    =     56.8 : 1.0
                V_farmer = True              ham : spam   =     50.6 : 1.0
                  V_lisa = True              ham : spam   =     47.3 : 1.0
                 V_susan = True              ham : spam   =     47.3 : 1.0
                  V_2005 = True             spam : ham    =     45.3 : 1.0
                   

In [17]:
goldlist_withoutStopwords,predictedlist_withoutStopwords=cross_validation_accuracy(num_folds, featuresets_withoutStopwords)

Each fold size: 600
0 0.975
1 0.9633333333333334
2 0.9633333333333334
3 0.9583333333333334
4 0.9716666666666667
mean accuracy 0.9663333333333334


In [18]:
# look at the first 30 examples
print(goldlist_withoutStopwords[:30])
print(predictedlist_withoutStopwords[:30])

['ham', 'ham', 'spam', 'spam', 'ham', 'spam', 'ham', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'ham', 'ham', 'spam', 'ham', 'spam', 'spam', 'ham', 'spam', 'spam', 'ham', 'spam']
['ham', 'ham', 'spam', 'spam', 'ham', 'spam', 'ham', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'ham', 'ham', 'spam', 'ham', 'spam', 'spam', 'ham', 'spam', 'spam', 'ham', 'spam']


In [33]:
cm_withoutStopwords = nltk.ConfusionMatrix(goldlist_withoutStopwords, predictedlist_withoutStopwords)
print(cm_withoutStopwords.pretty_format(sort_by_count=True, truncate=9))

     |         s |
     |    h    p |
     |    a    a |
     |    m    m |
-----+-----------+
 ham |<1410>  90 |
spam |   11<1489>|
-----+-----------+
(row = reference; col = test)



In [20]:
# or show the results as percentages
print(cm_withoutStopwords.pretty_format(sort_by_count=True, show_percents=True, truncate=9))

     |             s |
     |      h      p |
     |      a      a |
     |      m      m |
-----+---------------+
 ham | <47.0%>  3.0% |
spam |   0.4% <49.6%>|
-----+---------------+
(row = reference; col = test)



In [21]:
# call the function with our data
eval_measures(goldlist_withoutStopwords, predictedlist_withoutStopwords)

	Precision	Recall		F1
spam 	      0.993      0.943      0.967
ham 	      0.940      0.992      0.965


In [35]:
#let's try for POS tag 
# Function to get POS tags for each sentence in a list
def get_pos_tags(emaildocs):
    # Tokenize the text block into sentences
    sentences = nltk.sent_tokenize(' '.join(emaildocs))
    
    # Tokenize each sentence into words and apply POS tagging
    tagged_sentences = [nltk.pos_tag(nltk.word_tokenize(sentence)) for sentence in sentences]
    
    return tagged_sentences

# Applying the function to each list of sentences in the data structure
tagged_emailsents = [(get_pos_tags(email[0]), email[1]) for email in emaildocs]

# Now 'tagged_emailsents' contains the original sentences with their POS tags and spam/ham labels
print(tagged_emailsents[0])

([[('Subject', 'JJ'), (':', ':'), ('accrued', 'VBN'), ('thru', 'JJ'), ('nov', 'NN'), ('.', '.')], [('19', 'CD'), ('daren', 'NNS'), (',', ','), ('please', 'VB'), ('take', 'VB'), ('a', 'DT'), ('look', 'NN'), ('at', 'IN'), ('the', 'DT'), ('attached', 'JJ'), ('spreadsheet', 'NN'), (',', ','), ('we', 'PRP'), ('have', 'VBP'), ('over', 'RB'), ('500', 'CD'), (',', ','), ('000', 'CD'), ('mmbtu', 'NN'), ('as', 'IN'), ('unaccounted', 'JJ'), ('for', 'IN'), ('.', '.')], [('.', '.')], [('.', '.')], [('o', 'NN'), ("'", "''"), ('neal', 'JJ'), ('3', 'CD'), ('-', ':'), ('9686', 'CD')]], 'ham')


In [36]:
from collections import Counter

def pos_tag_features(tagged_sentences):
    # Flatten the list of sentences to get a list of (word, tag) tuples
    flat_list = [item for sublist in tagged_sentences for item in sublist]
    # Count the frequencies of the POS tags and words
    tag_counts = Counter(tag for word, tag in flat_list)
    word_counts = Counter(word.lower() for word, tag in flat_list if word.isalpha())
    # Combine the POS tag counts and word counts
    features = dict(tag_counts)
    features.update(word_counts)
    # Return the feature dictionary
    return features

In [37]:
posfeaturesets = [(pos_tag_features(email), label) for (email, label) in tagged_emailsents]
print(posfeaturesets[0])

({'JJ': 5, ':': 2, 'VBN': 1, 'NN': 5, '.': 4, 'CD': 5, 'NNS': 1, ',': 3, 'VB': 2, 'DT': 2, 'IN': 3, 'PRP': 1, 'VBP': 1, 'RB': 1, "''": 1, 'subject': 1, 'accrued': 1, 'thru': 1, 'nov': 1, 'daren': 1, 'please': 1, 'take': 1, 'a': 1, 'look': 1, 'at': 1, 'the': 1, 'attached': 1, 'spreadsheet': 1, 'we': 1, 'have': 1, 'over': 1, 'mmbtu': 1, 'as': 1, 'unaccounted': 1, 'for': 1, 'o': 1, 'neal': 1}, 'ham')


In [38]:
from sklearn.model_selection import train_test_split

# Test Train Split
pos_train_set, pos_test_set = train_test_split(posfeaturesets, test_size=0.2)

# Train classifier
posclassifier = nltk.NaiveBayesClassifier.train(pos_train_set)

# Test the classifier
posaccuracy = nltk.classify.accuracy(posclassifier, pos_test_set)
print(f"Classifier accuracy: {posaccuracy:.2f}")

# Most informative features
posclassifier.show_most_informative_features(5)

Classifier accuracy: 0.95
Most Informative Features
               forwarded = 1                 ham : spam   =    155.1 : 1.0
                     nom = 1                 ham : spam   =     80.7 : 1.0
                     bob = 1                 ham : spam   =     44.6 : 1.0
                    corp = 2                 ham : spam   =     41.9 : 1.0
              nomination = 1                 ham : spam   =     37.2 : 1.0


In [39]:
goldlist_postags,predictedlist_postags=cross_validation_accuracy(num_folds, posfeaturesets)

Each fold size: 600
0 0.9733333333333334
1 0.9583333333333334
2 0.955
3 0.9483333333333334
4 0.965
mean accuracy 0.96


In [40]:
# look at the first 30 examples
print(goldlist_postags[:30])
print(predictedlist_postags[:30])

['ham', 'ham', 'spam', 'spam', 'ham', 'spam', 'ham', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'ham', 'ham', 'spam', 'ham', 'spam', 'spam', 'ham', 'spam', 'spam', 'ham', 'spam']
['ham', 'ham', 'spam', 'spam', 'ham', 'spam', 'ham', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'ham', 'ham', 'spam', 'ham', 'spam', 'spam', 'ham', 'spam', 'spam', 'ham', 'spam']


In [41]:
cm_postags = nltk.ConfusionMatrix(goldlist_postags, predictedlist_postags)
print(cm_postags.pretty_format(sort_by_count=True, truncate=9))

     |         s |
     |    h    p |
     |    a    a |
     |    m    m |
-----+-----------+
 ham |<1384> 116 |
spam |    4<1496>|
-----+-----------+
(row = reference; col = test)



In [42]:
# or show the results as percentages
print(cm_postags.pretty_format(sort_by_count=True, show_percents=True, truncate=9))

     |             s |
     |      h      p |
     |      a      a |
     |      m      m |
-----+---------------+
 ham | <46.1%>  3.9% |
spam |   0.1% <49.9%>|
-----+---------------+
(row = reference; col = test)



In [43]:
# call the function with our data
eval_measures(goldlist_postags, predictedlist_postags)

	Precision	Recall		F1
spam 	      0.997      0.928      0.961
ham 	      0.923      0.997      0.958


In [46]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import precision_score, recall_score

In [45]:
#Get X and Y values:
# Instantiate the DictVectorizer to get X values
vectorizer = DictVectorizer(sparse=True)
# Convert the list of dictionaries to a sparse matrix
X = vectorizer.fit_transform([item[0] for item in featuresets_withoutStopwords])
y = [item[1] for item in featuresets_withoutStopwords]
#Test train split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [48]:
#Instantiate the model - start with LogisticRegression classifier as a base comparison model:
logreg= LogisticRegression()
#Fit the model
logreg.fit(X_train, y_train)
#Run Predictions
y_pred = logreg.predict(X_test)

# Calculate metrics
print("Test set accuracy: ", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

Test set accuracy:  0.975
              precision    recall  f1-score   support

         ham       1.00      0.95      0.97       303
        spam       0.95      1.00      0.98       297

    accuracy                           0.97       600
   macro avg       0.98      0.98      0.97       600
weighted avg       0.98      0.97      0.97       600



In [49]:
#Grid Search for Model Tuning
# Define the grid of values for tol and max_iter
tol = [.01,.001,.0001]
max_iter = [100, 150, 200]

# Create a dictionary where tol and max_iter are keys and the lists of their values are corresponding values
param_gridlog = dict(tol=tol, max_iter=max_iter)

# Instantiate GridSearchCV with the required parameters
grid_model = GridSearchCV(estimator=logreg, param_grid=param_gridlog, cv=5)

# Fit grid_model to the data
grid_model_result = grid_model.fit(X_train, y_train)

# Summarize results
best_score, best_params = grid_model_result.best_score_, grid_model_result.best_params_
print("Best: %f using %s" % (best_score, best_params))

# Extract the best model and evaluate it on the test set
best_log_model = grid_model_result.best_estimator_
best_log_model_result = best_log_model.fit(X_train, y_train)
y_best_log_pred = best_log_model_result.predict(X_test)
print("Test set accuracy: ", accuracy_score(y_test, y_best_log_pred))
print(classification_report(y_test, y_best_log_pred))
best_score, best_params = grid_model_result.best_score_, grid_model_result.best_params_
print("Best: %f using %s" % (best_score, best_params))

Best: 0.971667 using {'max_iter': 100, 'tol': 0.01}
Test set accuracy:  0.975
              precision    recall  f1-score   support

         ham       1.00      0.95      0.97       303
        spam       0.95      1.00      0.98       297

    accuracy                           0.97       600
   macro avg       0.98      0.98      0.97       600
weighted avg       0.98      0.97      0.97       600

Best: 0.971667 using {'max_iter': 100, 'tol': 0.01}


In [50]:
#Now a Random Forest Classifier
#Instantiate Model
rf = RandomForestClassifier(n_estimators=100, criterion='gini', min_samples_split=.1)
#Fit the model
rfresult = rf.fit(X_train, y_train) 
#Run prediction and evaluate
y_rfpred = rfresult.predict(X_test)
print("Test set accuracy:", accuracy_score(y_test, y_rfpred))
print(classification_report(y_test, y_rfpred))

Test set accuracy: 0.9666666666666667
              precision    recall  f1-score   support

         ham       1.00      0.93      0.97       303
        spam       0.94      1.00      0.97       297

    accuracy                           0.97       600
   macro avg       0.97      0.97      0.97       600
weighted avg       0.97      0.97      0.97       600



In [210]:
#Let's try GridSearch Cross Validation
params_rf = { 'n_estimators': [50, 100, 200, 500],
                'max_features': ['sqrt', 'log2'],
                'min_samples_split' : [.1,.12,.14,.16,.18,.2], 
                'criterion' :['gini', 'entropy']}
grid_rf = GridSearchCV(estimator=rf, param_grid=params_rf, cv= 5, refit=True)
# Fit grid_model to the data
grid_rf_model_result = grid_rf.fit(X_train, y_train)

# Summarize results
best_rf_score, best_rf_params = grid_rf_model_result.best_score_, grid_rf_model_result.best_params_
print("Best: %f using %s" % (best_rf_score, best_rf_params))

# Extract the best model and evaluate it on the test set
best_rf_model = grid_rf_model_result.best_estimator_
best_rf_model_result = best_rf_model.fit(X_train, y_train)
y_gscv_rf_pred = best_rf_model_result.predict(X_test)
print("Test set accuracy: ", accuracy_score(y_test, y_gscv_rf_pred))
print(classification_report(y_test, y_gscv_rf_pred))

' Uncomment this when ready to run\n#Let\'s try GridSearch Cross Validation\nparams_rf = { \'n_estimators\': [50, 100, 200, 500],\n                \'max_features\': [\'sqrt\', \'log2\'],\n                \'min_samples_split\' : [.1,.12,.14,.16,.18,.2], \n                \'criterion\' :[\'gini\', \'entropy\']}\ngrid_rf = GridSearchCV(estimator=rf, param_grid=params_rf, cv= 5, refit=True)\n# Fit grid_model to the data\ngrid_rf_model_result = grid_rf.fit(X_train, y_train)\n\n# Summarize results\nbest_rf_score, best_rf_params = grid_rf_model_result.best_score_, grid_rf_model_result.best_params_\nprint("Best: %f using %s" % (best_rf_score, best_rf_params))\n\n# Extract the best model and evaluate it on the test set\nbest_rf_model = grid_rf_model_result.best_estimator_\nbest_rf_model_result = best_rf_model.fit(X_train, y_train)\ny_gscv_rf_pred = best_rf_model_result.predict(X_test)\nprint("Test set accuracy: ", accuracy_score(y_test, y_gscv_rf_pred))\nprint(confusion_matrix(y_test, y_gscv_rf

In [51]:
#Now let's try Gradient Boosting
# Instantiate the Model
gb = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=42)

# Fit the model
gb.fit(X_train, y_train)

# Run prediction
y_gb_pred = gb.predict(X_test)

# Evaluate the model's performance
print("Test set accuracy:", accuracy_score(y_test, y_gb_pred))
print(classification_report(y_test, y_gb_pred))

Test set accuracy: 0.9633333333333334
              precision    recall  f1-score   support

         ham       0.98      0.94      0.96       303
        spam       0.94      0.98      0.96       297

    accuracy                           0.96       600
   macro avg       0.96      0.96      0.96       600
weighted avg       0.96      0.96      0.96       600



In [212]:
#And now with GridSearch
# Define the parameter grid to search
param_gb = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 4, 5],
    'subsample': [0.8, 0.9, 1.0]
}

# Instantiate the GridSearchCV object
gsgb = GridSearchCV(estimator=gb, param_grid=param_gb, cv=5, n_jobs=-1, verbose=1, scoring='accuracy')

# Fit the GridSearchCV object to the training data
gsgb.fit(X_train, y_train)

# Print the best parameters and the best score
print("Best parameters found: ", gsgb.best_params_)
print("Best cross-validated score: ", gsgb.best_score_)

# Extract the best estimator and evaluate it on the test set
best_gb = gsgb.best_estimator_
y_gsgb_pred = best_gb.predict(X_test)
print("Test set accuracy: ", accuracy_score(y_test, y_gsgb_pred))
print(classification_report(y_test, y_gsgb_pred))

' Uncomment this when ready to run\n#And now with GridSearch\n# Define the parameter grid to search\nparam_gb = {\n    \'n_estimators\': [100, 200, 300],\n    \'learning_rate\': [0.01, 0.1, 0.2],\n    \'max_depth\': [3, 4, 5],\n    \'subsample\': [0.8, 0.9, 1.0]\n}\n\n# Instantiate the GridSearchCV object\ngsgb = GridSearchCV(estimator=gb, param_grid=param_gb, cv=5, n_jobs=-1, verbose=1, scoring=\'accuracy\')\n\n# Fit the GridSearchCV object to the training data\ngsgb.fit(X_train, y_train)\n\n# Print the best parameters and the best score\nprint("Best parameters found: ", gsgb.best_params_)\nprint("Best cross-validated score: ", gsgb.best_score_)\n\n# Extract the best estimator and evaluate it on the test set\nbest_gb = gsgb.best_estimator_\ny_gsgb_pred = best_gb.predict(X_test)\nprint("Test set accuracy: ", accuracy_score(y_test, y_gsgb_pred))\nprint(confusion_matrix(y_test, y_gsgb_pred))\nprint("Precision:", precision_score(y_test, y_gsgb_pred, pos_label=\'spam\'))\nprint("Recall:",r

In [52]:
#SVM
#Instantiate
svm = SVC()
#Fit
svm.fit(X_train, y_train)
#Predict
y_svm_predict = svm.predict(X_test)

#Evaluate
print("Test set accuracy:", accuracy_score(y_test, y_svm_predict))
print(classification_report(y_test, y_svm_predict))

Test set accuracy: 0.9733333333333334
              precision    recall  f1-score   support

         ham       1.00      0.95      0.97       303
        spam       0.95      1.00      0.97       297

    accuracy                           0.97       600
   macro avg       0.97      0.97      0.97       600
weighted avg       0.97      0.97      0.97       600



In [214]:
# Instantiate the SVM model
svm = SVC()

# Define the parameter grid for GridSearchCV
param_grid = {
    'C': [0.1, 1, 10, 100],  # Regularization parameter
    'gamma': ['scale', 'auto'],  # Kernel coefficient
    'kernel': ['linear', 'rbf', 'poly']  # Specifies the kernel type to be used in the algorithm
}

# Create the GridSearchCV object
gssvm = GridSearchCV(estimator=svm, param_grid=param_grid, cv=5, scoring='accuracy')

# Fit the grid search to the data
gssvm.fit(X_train, y_train)

# Get the best parameters and the best score
best_parameters = gssvm.best_params_
best_score = gssvm.best_score_

# Print the best parameters and the best score
print("Best parameters found: ", gssvm.best_params_)
print("Best cross-validated score: ", gssvm.best_score_)

# Extract the best estimator and evaluate it on the test set
best_svm = gssvm.best_estimator_
y_gssvm_pred = gssvm.predict(X_test)
print("Test set accuracy: ", accuracy_score(y_test, y_gssvm_pred))
print(classification_report(y_test, y_gssvm_pred))

'Uncomment this when ready to run\n# Instantiate the SVM model\nsvm = SVC()\n\n# Define the parameter grid for GridSearchCV\nparam_grid = {\n    \'C\': [0.1, 1, 10, 100],  # Regularization parameter\n    \'gamma\': [\'scale\', \'auto\'],  # Kernel coefficient\n    \'kernel\': [\'linear\', \'rbf\', \'poly\']  # Specifies the kernel type to be used in the algorithm\n}\n\n# Create the GridSearchCV object\ngssvm = GridSearchCV(estimator=svm, param_grid=param_grid, cv=5, scoring=\'accuracy\')\n\n# Fit the grid search to the data\ngssvm.fit(X_train, y_train)\n\n# Get the best parameters and the best score\nbest_parameters = gssvm.best_params_\nbest_score = gssvm.best_score_\n\n# Print the best parameters and the best score\nprint("Best parameters found: ", gssvm.best_params_)\nprint("Best cross-validated score: ", gssvm.best_score_)\n\n# Extract the best estimator and evaluate it on the test set\nbest_svm = gssvm.best_estimator_\ny_gssvm_pred = gssvm.predict(X_test)\nprint("Test set accuracy